In [8]:
import numpy as np
import csv


from modules.cas_TS import RecSys


import pandas as pd
import matplotlib.pyplot as plt
import pickle

In [9]:
items_loc = 'csv_files/items.csv'
data_loc = 'users_data/users_data.csv'
parameters_loc = 'parameters/parameters.pickle'

In [10]:
K = 6 #number of recommendations per user
n = 25 #total number of days/rounds the simulations run for
alpha = 0 #diversity factor
number_of_users = 100 #number of users per day
from_scratch = 0 #indicator specifying wether the training should start from scratch or continue from the latest trained parameteres





In [11]:
items = pd.read_csv(items_loc)
titles = list(items['IDENTIFIER'])
ctr = {t:0.01*np.random.randint(1,K) for t in titles}
click_probabilities = np.array(list(ctr.values()))
L = items.shape[0] #number of items



In [12]:
regrett = []
repeatt = []
Rec = RecSys(items,K, L, alpha)

In [13]:
for day in range(1, n+1):
    if day>1:
        data = pd.read_csv(data_loc)
    else:
        data = {}
  
    Rec.train(parameters_loc, data, from_scratch)
    repeatt.append(Rec.repeat)
        
    header = ['selected']+[r'choice{}'.format(i+1) for i in range(K)]  
    with open(data_loc, 'w') as file:
        writer = csv.writer(file)
        writer.writerow(header)

    

    
    regs = 0    
    
    for user in range(number_of_users):
        recommendations = Rec.predict(parameters_loc)
        Rec.make_recommendation_and_save_new_data(click_probabilities, recommendations, data_loc)
        regs = regs+Rec.regret
        
    regrett.append(regs)
        


In [17]:
#ploting regret curve
q = 0
p = n
regret = np.array([regrett[day] for day in range(n)])
ff = np.array([sum(regret[0:t])/number_of_users for t in range(1,n+1)])
plt.plot(ff[q:p]/np.arange(q+1,p+1))
plt.xlabel(r'Round, $t$')
plt.ylabel(r'Per-step regret, $\frac{R_t}{t}$')

In [16]:
#plotting the repeatition curve
plt.plot(np.array(repeatt))
plt.ylim(-0.1,K+0.1)
plt.xlabel(r'Round, $t$')
plt.ylabel(r'Repetition$')